In [40]:
import pandas as pd
import numpy as np
import pickle
import os
import itertools

import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

In [4]:
data = pd.read_csv("../../data/data_cleaned_sentences_phases_2020-04-16.csv", sep='|', converters={'sentences': pd.eval})

In [5]:
texts = data.loc[0, "sentences"]
texts

['sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı',
 'ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi',
 'polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı',
 'rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi',
 'polisin çalışması devam ediyor']

In [7]:
texts = data.loc[0, "sentences"]
texts

['sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı',
 'ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi',
 'polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı',
 'rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi',
 'polisin çalışması devam ediyor']

In [12]:
texts_all = " [SEP] ".join(texts)
texts_all

'sahte polislerin kuryesi yakalandı antalyada kendi lerini polis olarak ta nıtarak dolandırıcılık yapan çeteye kuryemi bllik yapan suriyeli bana mustafa banka şubesinde düzenlenen operasyonla yakalandı [SEP] ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi [SEP] polisin kendisini beklediğinden habersiz olan mustafa kaçmaya çalışırken yakalandı [SEP] rana mustafanın ifadesi doğrultusunda iki çete üyesinin yeri tespit edildi [SEP] polisin çalışması devam ediyor'

In [7]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

Downloading: 100%|██████████| 59.0/59.0 [00:00<00:00, 111kB/s]
Downloading: 100%|██████████| 386/386 [00:00<00:00, 847kB/s]
Downloading: 100%|██████████| 1.18M/1.18M [00:00<00:00, 1.39MB/s]


In [24]:
tokenizer.cls_token_id, tokenizer.sep_token_id

(2, 3)

In [17]:
texts[1]

'ye bankadan bintl çektiren çete üyeleri parayı alması için mustafayı bankaya gönderdi'

In [37]:
tokenizer("[PAD]", truncation=True, padding='max_length', max_length=21)

{'input_ids': [2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [41]:
a = tokenizer(texts, truncation=True, padding=True)
b = [list(itertools.chain.from_iterable(a))[:-1]]
a, b

({'input_ids': [[2, 8449, 24635, 16217, 2446, 12088, 58520, 2676, 40082, 3574, 2095, 2058, 6512, 78175, 22704, 3149, 15279, 2189, 16217, 2932, 5735, 2065, 3149, 12874, 2789, 4190, 3730, 12305, 2386, 54080, 2956, 31083, 12088, 3], [2, 2683, 27631, 116288, 1019, 115973, 2280, 15279, 1011, 84087, 2023, 8546, 7641, 8059, 4190, 2426, 18517, 41569, 1971, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 11821, 5267, 29577, 53677, 18387, 2120, 4190, 21783, 2282, 52565, 4763, 12088, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 64557, 4190, 16853, 11649, 119506, 2537, 15279, 1011, 84087, 5326, 4673, 3985, 4341, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 11821, 52565, 2094, 2524, 2992, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [9]:
tokenizer

PreTrainedTokenizerFast(name_or_path='dbmdz/bert-base-turkish-128k-uncased', vocab_size=128000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})